In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# Read recipe inputs
alert_PREDICT_EXPLANATION_prepared = dataiku.Dataset("ALERT_PREDICT_EXPLANATION_prepared")
alert_PREDICT_EXPLANATION_prepared_df = alert_PREDICT_EXPLANATION_prepared.get_dataframe()

In [ ]:
##the data base
alert_PREDICT_EXPLANATION_prepared_df.head()

In [ ]:
### Computing the sum of shapley values for each row
mean_ln_odd_pred=alert_PREDICT_EXPLANATION_prepared_df['ln_odd_pred_prob'].mean()
alert_PREDICT_EXPLANATION_prepared_df['Sum_shapley']=alert_PREDICT_EXPLANATION_prepared_df['ln_odd_pred_prob']-mean_ln_odd_pred
alert_PREDICT_EXPLANATION_prepared_df.head()

In [ ]:
## Main columns of interest
cols=[0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
X_all=alert_PREDICT_EXPLANATION_prepared_df[alert_PREDICT_EXPLANATION_prepared_df.columns[cols]]
X_all=X_all.set_index('CERFRANCE')
X_all.head()

In [27]:
### Function to transform shapley values into predicted probability variation

def shap2deltaprob(features,
                   shap_df,
                   shap_sum,
                   probas,
                   func_shap2probas):
    '''
    map shap to Δ probabilities
    --- input ---
    :features: list of strings, names of features
    :shap_df: pd.DataFrame, dataframe containing shap values
    :shap_sum: pd.Series, series containing shap sum for each observation
    :probas: pd.Series, series containing predicted probability for each observation
    :func_shap2probas: function, maps shap to probability (for example interpolation function)
    --- output ---
    :out: pd.Series or pd.DataFrame, Δ probability for each shap value
    '''
    # 1 feature
    if type(features) == str or len(features) == 1:
        return probas - (shap_sum - shap_df[features]).apply(func_shap2probas)
    # more than 1 feature
    else:
        return shap_df[features].apply(lambda x: shap_sum - x).apply(func_shap2probas)\
                .apply(lambda x: probas - x)

In [28]:
### Inputs for the function defined above
shap_sum=alert_PREDICT_EXPLANATION_prepared_df['Sum_shapley']
cols1=[0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
shap_df=alert_PREDICT_EXPLANATION_prepared_df[alert_PREDICT_EXPLANATION_prepared_df.columns[cols1]]
probas=alert_PREDICT_EXPLANATION_prepared_df['proba_1']
shap_df=shap_df.set_index('CERFRANCE')

In [29]:
probas=pd.Series(alert_PREDICT_EXPLANATION_prepared_df['proba_1'].tolist(),index=X_all.index)
shap_sum=pd.Series(alert_PREDICT_EXPLANATION_prepared_df['Sum_shapley'].tolist(),index=X_all.index)

In [30]:
# build interpolation function to map shap into probability
shap_sum_sort = shap_sum.sort_values()
probas_sort = probas[shap_sum_sort.index]

intp = interp1d(shap_sum_sort,
                probas_sort,
                bounds_error = False,
                fill_value = (0, 1))

In [ ]:
### Regarder l'allure de la courbe
plt.scatter(shap_sum_sort, probas_sort,color='blue',zorder=2)

In [32]:
# show Δ probabilities
temp = shap2deltaprob(X_all.columns.tolist(),
                      shap_df,
                      shap_sum,
                      probas,
                      func_shap2probas=intp)

In [ ]:
temp['CERFRANCE']=temp.index
temp['Shap_sum']=shap_sum
temp.head()

In [ ]:
alert_PREDICT_EXPLANATION_prepared_df.head()

In [ ]:
col_to_norm=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
temp2=temp[temp.columns[col_to_norm]]

temp_normalized=temp2.apply(lambda x:(x-x.mean())/x.std())

temp_normalized.describe()

In [ ]:
temp_norm2=X_all.apply(lambda x:(x-x.mean())/x.std())
temp_norm2.head()

In [ ]:
temp_fillna=X_all.fillna(0)
temp_fillna.head()
temp_fillna_norm=temp_fillna.apply(lambda x:(x-x.mean())/x.std())
temp_fillna_norm['CERFRANCE']=temp_fillna_norm.index
temp_fillna_norm['Shap_sum']=shap_sum
temp_fillna_norm.head()

In [40]:
# Write recipe outputs
alert_PREDICT_EXPLANATIONS_DATA_df =temp_fillna_norm # For this sample code, simply copy input to output
alert_PREDICT_EXPLANATIONS_DATA = dataiku.Dataset("ALERT_PREDICT_EXPLANATIONS_DATA")
alert_PREDICT_EXPLANATIONS_DATA.write_with_schema(temp_fillna_norm)

57 rows successfully written (zHISg9I1LN)
